In [ ]:
# Part 0: Environment Setup & Drive Mounting
from google.colab import drive
import os

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Define the project root directory in your Drive
PROJECT_ROOT = "/content/drive/My Drive/ECE253_Project"

# 3. Define sub-directories for organized storage
DIRS = {
    "GT": os.path.join(PROJECT_ROOT, "1_GroundTruth"),      # Folder for Sharp images
    "BLUR": os.path.join(PROJECT_ROOT, "2_InputBlur"),      # Folder for Blurred images
    "RL_OUT": os.path.join(PROJECT_ROOT, "3_Output_RL"),    # Folder for RL results
    "NAF_OUT": os.path.join(PROJECT_ROOT, "4_Output_NAFNet"), # Folder for NAFNet results
    "METRICS": os.path.join(PROJECT_ROOT, "Metrics")        # Folder for CSV files
}

# 4. Create directories if they don't exist
for k, v in DIRS.items():
    os.makedirs(v, exist_ok=True)

print(f"✅ Working directory ready: {PROJECT_ROOT}")

Mounted at /content/drive
✅ Working directory ready: /content/drive/My Drive/ECE253_Project


In [ ]:
# Part 1: Data Generation
import cv2
import numpy as np
from datasets import load_dataset
from tqdm import tqdm
import os

# --- 1. Define Motion Blur Kernel ---
def make_motion_kernel(size, angle):
    # Create a line kernel
    k = np.zeros((size, size), dtype=np.float32)
    k[(size - 1) // 2, :] = np.ones(size, dtype=np.float32)

    # Rotate the kernel
    M = cv2.getRotationMatrix2D((size / 2 - 0.5, size / 2 - 0.5), angle, 1)
    k = cv2.warpAffine(k, M, (size, size))

    # Normalize kernel so sum is 1
    k = k * (1.0 / np.sum(k))
    return k

# Parameters
KERNEL_SIZE = 15
ANGLE = 0
psf = make_motion_kernel(KERNEL_SIZE, ANGLE)

# Save the kernel to Drive (Critical for Part 2)
kernel_path = os.path.join(PROJECT_ROOT, "motion_kernel.npy")
np.save(kernel_path, psf)
print(f">>> Motion kernel saved to: {kernel_path}")

# --- 2. Download and Process Images ---
TARGET_COUNT = 100
print(f">>> Generating {TARGET_COUNT} image pairs...")

# Load dataset (Streaming mode to save RAM)
ds = load_dataset("Francesco/road-traffic", split="train", streaming=True)
count = 0

for item in tqdm(ds, desc="Processing Images"):
    if count >= TARGET_COUNT: break

    # Check if image data exists
    if 'image' not in item: continue

    image = item['image'].convert('RGB')

    # Skip small images
    if image.size[0] < 256 or image.size[1] < 256: continue

    filename = f"traffic_{count:04d}.jpg"

    # A. Save Ground Truth (GT)
    img_gt = np.array(image)
    # Convert RGB to BGR for OpenCV saving
    cv2.imwrite(os.path.join(DIRS["GT"], filename), cv2.cvtColor(img_gt, cv2.COLOR_RGB2BGR))

    # B. Synthesize Blur (Input)
    img_float = img_gt.astype(np.float32) / 255.0
    img_blur = np.zeros_like(img_float)

    # Apply kernel to each channel (R, G, B)
    for i in range(3):
        img_blur[:,:,i] = cv2.filter2D(img_float[:,:,i], -1, psf)

    # Save Blurred Image
    cv2.imwrite(os.path.join(DIRS["BLUR"], filename), cv2.cvtColor((img_blur * 255).astype(np.uint8), cv2.COLOR_RGB2BGR))

    count += 1

print("✅ Part 1 Complete: Data and Kernel saved to Google Drive.")

>>> Motion kernel saved to: /content/drive/My Drive/ECE253_Project/motion_kernel.npy
>>> Generating 100 image pairs...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

Processing Images: 100it [00:11,  8.66it/s]

✅ Part 1 Complete: Data and Kernel saved to Google Drive.
